# Controller Playground

All credit for the beautiful markdown goes to Jamsheed

In [1]:
# Bring in the MSI object
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from simulation import state_base, params_base, vesting_schedule_translate

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np
from pprint import pprint

ms = load_from_json(deepcopy(math_spec_json))
msi = ms.build_implementation(params_base)
state, params = msi.prepare_state_and_params(state_base, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])

Placeholder has no control action options
Mezzanine Wiring Passthrough has no policy options
No python implementation for Increment Block Number Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Increment Block Number Mechanism
No python implementation for Update Historical Mined Ratio Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Mined Ratio Mechanism
No python implementation for Update Historical Qi Hash Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Qi Hash Mechanism
No python implementation for Update Historical Quai Hash Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Update Historical Quai Hash Mechanism
No python implementation for Log Simulation Data Mechanism. To fix this, go to Implementations/Python/Mechanisms and add Log Simulation Data Mechanism
No python implementation for Set K Mechanism. To fix this, go to Implementations/Python/Mechanisms

## Assumptions Added

Asset 1 is Quai, asset 2 is Qi

## Miner Probability

In what follows:
$$
  c_i = 
  \begin{cases}
    1 & \text{if token 1 is chosen} \\
    0 & \text{if token 2 is chosen}.
  \end{cases}
$$

Miner choices $c_i$ are assumed to be independently distributed such that for a block at height $i$,
$$
  p_i = \Pr(c_i = 1 | r_{i1}, r_{i2}, d_i ) := \frac{1}{1 + \exp(- \pmb{\beta}'\mathbf x_i) },
$$
where $\mathbf x_i$ is a set of features and $\pmb \beta$ their associated weights. It may be that the first such feature is $1$, so that the first weight is an intercept/'bias' term. Note that the linear term $\pmb{\beta}' \mathbf x$ is consistent with an interpretation of the above as coming from a latent variable/random utility model of the miner.

Given the data set $z_k$, maximum likelihood estimation yields estimates $\hat{\pmb{\beta}}$.

In [2]:
import numpy as np

def miner_probability(beta, x):
    return 1 / (1+ np.exp(beta.dot(x)))

### Objective: stability via indifference

The controller seeks to stabilize an imputed value of hashpower (difficulty) by adjusting the proposed block rewards so that the miner would have been _indifferent_ between receiving an award in _qi_ (token 1) or _quai_ (token 2). The interpretation of this is that _deviations from indifference reveals that one token is more valuable than the other_. In the case that one token (_qi_) is to reflect the value of hashpower (difficulty), indifference is a _reference_ or _focal_ point from which the value of hashpower may be observed from miner decisions.

Indifference is when $p_i = 0.5$. Given $\hat{\pmb{\beta}}$, it is clear that the _invariant surface_ of features satisfies
$$
  \hat{\pmb{\beta}}' \mathbf x \equiv 0.
$$

Refining this further requires a definition of the features $\mathbf x$.

### A simple example

The simplest example is where $\mathbf x_i = (1, x_i) := (1, d_i/\log_2(d_i))$. In this case the invariant surface above yields a value $d_i = d^\star$ such that
$$
  \frac{d^\star}{\log_2(d^\star)} = -\frac{\hat{\beta_0}}{\hat{\beta_1}}.
$$

This is the difficulty level that would have to obtain in order for a miner to be (on average) indifferent between selecting token 1 and token 2. In this case define $x^\star(\hat{\pmb{\beta}}) = d^\star / \log_2(d^\star)$ (we will sometimes drop the dependence of $x^\star$ upon $\hat{\pmb{\beta}}$ for brevity in what follows, but it is important always to recall that $x^\star$ is derived from the _estimation problem_ the controller performs in finding a miner's indifference point).

[It is worth noting here that provided $d_i > e$, $\frac{dx_i}{d(d_i)} > 0$, i.e. increasing difficulty $d_i$ will increase $x_i$ and hence increase $p_i$ from the logistic expression above. There is thus a weak restriction on $d_i$ under this approach.]

Beta is given, we have true population. Build functionality to draw betas from distribution.

Add to the state:

Population Beta -> Can change in terms of being pulled from a random distribution

Sample/Estimated Beta -> Updated with batch learning

In [3]:
from math import log
def prepare_x(state, params):
    d1 = state["Block Difficulty"]
    d2 = log(state["Block Difficulty"], params["Quai Reward Base Parameter"])
    x = np.array([1, d1/d2])
    return x
x = prepare_x(state, params)
print(x)

[  1.         478.05920258]


### Loss function

Although $d_i^\star$ is not part of $z_k$ (or is v. unlikely to be), the form of the controller impact is known from
$$
  x^\star = \left ( \frac{u_2}{u_1} \right )\left ( \frac{r_{1}}{r_{2}} \right ),
$$
where $r_1$ and $r_2$ are unspecified proposed block rewards.

Letting $r_{1}/r_{2}$ be given as a _parameter_ $R$, this defines a second invariant surface:
$$
R u_2 = x^\star  u_1.
$$

The controller seeks to select $(u_1^\star, u_2^\star)$ to satisfy this invariant, which is an implicit loss function since this can equivalently be written
$$
  \min_{u_1,u_2} \ell(x^\star, u_1, u_2; R) := R u_2 - x^\star u_1
$$
such that $\ell(\cdot) \geq 0$.

Since the loss function is parameterized by $R$, a key question is what this value should be. As the latest available information of the proposed block rewards is $r_{1k}/r_{2k}$ for prime block $k$, this could be used as an estimate of $R$. Alternatively, smoothing of values from $z_k$, e.g. EWMA of the ratios $r_{1i}/r_{2i}$, could be used.

Under this interpretation, either $u_1$ or $u_2$ may be set as a _numeraire_, reducing the required control space by one dimension. This reduction is because of the simple form of $\mathbf x_i$ here, but more features will generally require additional restrictions on both $u_1$ and $u_2$ that must come from elsewhere (the dimension of the invariant surface grows in that case).

In [4]:
R = state["Metrics"]["Current Block Reward Ratio Metric"](state, params, [])
print(R)

0.002091791130897207


### Gain

#### Gain as a control variable

A hypothesis from Aug 21st is that a _gain_ is required that is dependent upon the average block difficulty $D$, and an initial supposition was to set $u_2 = g(D)$ for some function $g$. 

The role of the gain is to set the scale of the proposed block rewards $r_1$ and $r_2$, adjusting that scale in response to changes in $D$. This is a 'black box' approach that does not change the approach above (since only the ration $u_1/u_2$ matters). However, it does not have a theoretical basis for its introduction, rather acting as an _ad hoc_ adjustment parameter.

It is likely that for controller stability at least $g'(D) < 0$ (and perhaps $g''(D) > 0$) everywhere, but this is not confirmed.

#### Gain as a reward ratio

An alternative that may provide more insight into scaling is to treat the unspecified parameter $R$ as a function of $D$, allowing that to change in an consistent fashion relative to the protocol. This has the advantage of 'closing the loop' on $R$ and providing a response to average difficulty $D$. Since $R$ is meant to be an expression of a ratio of proposed block rewards, one may adopt the definition of $R(D)$ as the ratio of proposed block rewards that _would have obtained_ if $D$ had been the block difficulty under the previous controller parameters, i.e.:
$$
  R(D) := \frac{u_{k1}}{u_{k2}}x(D),
$$
where $\mathbf u_k := (u_{k1}, u_{k2})$ is the vector of controller parameters from previous prime block $k$, and
$$
  x(D) := \frac{D}{\log_2(D)},
$$
i.e. that ratio of difficulties that would have obtained from difficulty level $D$.

Given this, the optimal controller parameter update for $u^\star_1$ from the loss function is (from above):
$$
  u_1^\star = \frac{u_2^\star}{u_{k2}}u_{k1}\frac{x(D)}{x^\star(\hat{\pmb{\beta}})}.
$$

If (as before) token 2 acts as a numeraire, then $u^\star_2 = u_{k2}$. Letting $u_1^\star = u_{k1} + \Delta u_1$, this can be rearranged to yield the update "delta" for the controller paramter $u_1$:
$$
  \Delta u_1 = \left ( \frac{x(D)}{x^\star(\hat{\pmb{\beta}})} - 1 \right )u_{k1}. \qquad \qquad (*)
$$

This has an appealing interpretation for the controller: when the function of average difficulty $x(D)$ falls below the estimated value $x^\star(\hat{\pmb{\beta}})$, the controller parameter $u_1$ is adjusted proportionally downwards, so that the proposed block rewards, at the indifference difficulty level $d^\star$, reflect the block ratio $R(D)$ for the average difficulty. **The controller is thus acting to _stabilize_ the block reward ratio around the average difficulty of the system**. Similarly, when $x(D) > x^\star(\hat{\pmb{\beta}})$ the controller parameter $u_1$ is adjusted upwards.

In this setup the control parameter $u_2$ retains its use as a numeraire, i.e. it is set to a constant $\bar u$ derived to satisfy initial condition requirements and remains unchanged thereafter.

:::info
A manual "attenuation" parameter could also be added here, to adjust the sensitivity of $\Delta u_1$ to $x(D)/x^\star(\hat{\pmb{\beta}})$. In that case $(*)$ above is slightly modified to
$$
  \Delta u_1 = \alpha \left ( \frac{x(D)}{x^\star(\hat{\pmb{\beta}})} - 1 \right )u_{k1}. \qquad \qquad (**)
$$
for an exogenous parameter $\alpha \in (0,1]$. It is not expected that this manual attenuation is required, but may be used if a slower convergence to the indifference point is desired.
:::

$$
  x^\star(\hat{\pmb{\beta}}) = \frac{d^\star}{\log_2(d^\star)} = -\frac{\hat{\beta_0}}{\hat{\beta_1}}.
$$

1. All that needs to be done on the controller update function is bring in x_b_star all else fine
2. For the beta hats need to come to consensus on how beta hat gets updated

In [5]:
def controller_gain_function(state, params, alpha, R):
    state["K Quai"]
    D = state["Block Difficulty"]
    d1 = D
    d2 = log(D, params["Quai Reward Base Parameter"])
    x_d = d1 / d2
    x_b_star = 

def gain_ratio_reward(state, params, D):
    u_ratio = state["K Quai"] / state["K Qi"]
    d1 = D
    d2 = log(D, params["Quai Reward Base Parameter"])
    return u_ratio * (d1 / d2)

# This could either be the current reward ratio or based on proposed
R = gain_ratio_reward(state, params, 1000)
#R = state["Metrics"]["Current Block Reward Ratio Metric"](state, params, [])
print(R)

# Parameter
alpha = .5

k_qi_delta = controller_gain_function(state, params, alpha, R)
print(k_qi_delta)


SyntaxError: invalid syntax (3211355643.py, line 7)

### State update

What is the response of the miner to the new controller state? Let the next (non-prime) block $i$ have difficulty $d_i$. If $d_i < d^\star$ then $x_i < x^\star$ and so $p_i$, the probability a miner accepts token 1, must be less than 1/2. We see that _the valuation of token 1 has decreased in expectation when difficulty has decreased, as it is the miner decision that conveys difficulty value_.

Similarly, if $d_i > d^\star$ then $x_i > x^\star$, and so $p_i > 1/2$, i.e. _the valuation of token 1 has increased in expectation when difficulty has increased_.

The interpretation links the valuation of difficulty to the value of the _qi_ token--in this case the _quai_ token acts as a _numeraire_.
